In [1]:
import os
import sys
sys.path.append('/home/shamil/AILab/Rosreest')

In [2]:
import datetime
import json
import re
import fnmatch
from utils.preprocess import pycococreatortools
from utils.preprocess.tif_process import *
from utils.preprocess.slice_dataset import slice
import shutil
from pathlib import Path
from utils.preprocess.split_channels import split_channels

In [3]:
# root path for saving the tif and shp file.
root = Path(r'/home/shamil/AILab/tmp_images')
img_path = root / 'Raster'
shp_path = root / 'Razmetka'

clip_size = 3800

clip_folder = root / f"clip{clip_size}"

if clip_folder.is_dir():
    shutil.rmtree(clip_folder)

clip_folder.mkdir()

bands = clip_folder / 'bands'
bands.mkdir(exist_ok=True)

ground_truth = clip_folder / 'ground_truth'
ground_truth.mkdir(exist_ok=True)

with open(clip_folder / 'score_thresholds.json', 'w') as file:
    file.write(json.dumps({"f_score": 0.9}))

# root path for saving the mask.
dataset_dir = clip_folder
image_dir = dataset_dir /  "uchastok_2021"
annotations_dir = dataset_dir / "annotations"

## Configs (do not change)


In [4]:

INFO = {
    "description": "ZemleUchastki",
    "url": "",
    "version": "0.1.1",
    "year": 2021,
    "contributor": "Ibragim, Shamil",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "",
        "url": ""
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'uchastok',
        'supercategory': 'land',
    },
]


In [5]:

def save_clip(image_name: str):
    image_path = img_path / image_name

    tiff = GeoTiff(str(image_path))
    image_id = image_name.split(".")[0]

    tiff.clip_tif_and_shapefile(
        clip_size=clip_size,
        begin_id=0,
        shapefile_path=str(shp_path / image_id / (image_id + ".shp")),
        out_dir=str(clip_folder)
    )

    print(f"Saving results to: {str(clip_folder)}")

save_clip(image_name='Stolbische-1_N-39-015-(008-j)_COFP.tif')

create dir /home/shamil/AILab/tmp_images/clip4096/annotations
create dir /home/shamil/AILab/tmp_images/clip4096/uchastok_2021
Saving results to: /home/shamil/AILab/tmp_images/clip4096


In [6]:
leave_image_name ='0.tif'

In [7]:
def clean_dataset_folder(exclude_im_names: list[str]):
    """
    Remove not needed annotations / images
    """
    exclude_annotations = tuple(map(lambda x: x.split('.')[0] + "_", exclude_im_names))
    exclude_images = tuple(exclude_im_names)
    print(f"exclude_images: {exclude_images}")
    for path in annotations_dir.rglob("**/*"):
        if path.is_file():
            name = path.name
            if not name.startswith(exclude_annotations):
                path.unlink()

    for path in image_dir.rglob("**/*"):
        if path.is_file():
            name = path.name
            if not name.startswith(exclude_images):
                path.unlink()

clean_dataset_folder(exclude_im_names=[leave_image_name])

exclude_images: ('0.tif',)


In [8]:
# Generating json with annotations


def generate_json():
    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }

    image_id = 1
    segmentation_id = 1

    # filter for jpeg images
    image_path = clip_folder / 'uchastok_2021' / leave_image_name
    image = Image.open(image_path)
    image_info = pycococreatortools.create_image_info(
            image_id, os.path.basename(image_path.name), image.size)

    coco_output["images"].append(image_info)

    for annotation_path in annotations_dir.rglob("**/*"):
        if annotation_path.is_file():
            print(str(annotation_path))
            annotation_filename, image_filename = str(annotation_path), str(image_path)

            class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]

            category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
            binary_mask = np.asarray(Image.open(annotation_filename)
                                     .convert('1')).astype(np.uint8)

            annotation_info = pycococreatortools.create_annotation_info(
                segmentation_id, image_id, category_info, binary_mask,
                image.size, tolerance=2)

            if annotation_info is not None:
                coco_output["annotations"].append(annotation_info)

            segmentation_id = segmentation_id + 1

    with open(ground_truth / "annotations.json", 'w') as file:
        file.write(json.dumps(coco_output))

generate_json()


FileNotFoundError: [Errno 2] No such file or directory: '/home/shamil/AILab/tmp_images/clip4096/uchastok_2021/0.tif'

In [ ]:
# Slicing image to 3 channels

split_channels(
    _image_path=image_dir / leave_image_name,
    _dst=bands
)





